# Python Overview

This chapter contains an overview to Python. It covers the following:

1. The Assignment Operation
2. Instantiation
3. Methods
4. Expressions and Operators
5. Functions
6. Exceptions
7. Iterable Type
8. Scopes and Namespaces
9. Loading Modules

For each of these, I've added my notes and some examples to help see what Python is doing.

Python is an <u>interpreted language</u>. 
* The interpreter receives code from a script and executes it.
* The interpreter is run in interactive mode by default (i.e. when typing python into the terminal).
* Python is an <u>object oriented language</u> and <u>Classes</u> form the basis for all data types.


## 1. The Assignment Operation

* Each <u>identifier</u> is associated with the memory address of the object to which it refers (or to the special object None)
* Python is <u>dynamically typed</u>. Do not need to declare the data type of the identifier
* <u>Alias</u> an object by assigning a second identifier to it


Lets assign an _Identifier/name_ gets assigned an _object_.

In [1]:
temperature = 98.6

We can _alias_ an object by assigning a second identifier to it:

In [2]:
original = temperature   # alias

We can see that these point to the same object

In [3]:
id(original) == id(temperature)

True

We can _break_ this alias

In [4]:
temperature = temperature + 5.0
id(original) == id(temperature)

False

## Instantiation

In [5]:
temperature = 98.6 # literal form
temperature2 = float(98.6) # calling constructor method

## Expressions and Operators

`is` (alias) versus `==` (equivalence)

We can see that: `is` => `==`

In [6]:
a = [10]
b = a # a, b alias
c = [10]

print(a is b) # true
print(a == b) # true
print(a is c) # false
print(a == c) # true

True
True
False
True


`x+=y` versus `x = x + y`

In [7]:
alpha = [1, 2, 3]
beta = alpha                        # an alias for alpha
beta += [4, 5]                      # mutates the original list by extending it
beta = beta + [6, 7]                # reassigns beta to a new list (i.e. broke the alias)
print('alpha = {}'.format(alpha))
print('beta = {}'.format(beta))

alpha = [1, 2, 3, 4, 5]
beta = [1, 2, 3, 4, 5, 6, 7]


Note that lists are _mutable classes_. The behavior is different if the class is not mutable (e.g. tuples)

In [8]:
alpha = (1, 2, 3)
beta = alpha                        # an alias for alpha
beta += (4, 5)                      # reassigns beta to a new tuple (1, 2, 3, 4, 5)
print('alpha = {}'.format(alpha))   # will be (1, 2, 3) NOT (1, 2, 3, 4, 5)
print('beta = {}'.format(beta))

alpha = (1, 2, 3)
beta = (1, 2, 3, 4, 5)


Although tuples are themselves immutable, their contents can be mutatable datatypes (and hence can mutate)!

In [9]:
alpha = ([1,2,3], [4,5,6])
beta = alpha                              # alias for alpha

alpha[0][1] += 10                         # changing a mutable element of alpha
print('alpha = {}'.format(alpha))
print('beta = {}'.format(beta))           # notice both have changed!

alpha = ([1, 12, 3], [4, 5, 6])
beta = ([1, 12, 3], [4, 5, 6])


Operations depend on data type (are overloaded)

In [10]:
print(3*'hi') # uses the * implementation of string vs int
print('hi'*3)

hihihi
hihihi


Can make your own class and overload operations like `+`, `len`, ...

In [11]:
class vector():
    def __init__(self, d):
        self._coords = [0]*d
    def __len__(self):
        return len(self._coords)
    def __getitem__(self, i):
        return self._coords[i]
    def __add__(self, other):
        for i in range(len(self)):
            self[i] += other[i]
    def __setitem__(self, j, val):
        self._coords[j] = val
    def __repr__(self):
        to_return = '<'
        for i in range(len(self)):
            to_return += ' {} '.format(self[i])
        to_return += '>'
        return to_return

In [12]:
x = vector(4)
x[0] = 1
x[1] = 2
x[2] = 3
x[3] = 5

In [13]:
x + [1,2,3,4]
print(x)

< 2  4  6  9 >


In [14]:
try:
    [1,2,3,4] + x # Python uses left operand definition of +
    print(x)
except TypeError:
    print('could not perform operation')

could not perform operation


we can iterate our class implicitly since we implemented `__len__` and `__getitem__` (see next section for what this is)

In [15]:
total = 0
for entry in x:
    total += entry
print(total)

21


## Iterators and Iterables

1. _iterable_ 
    * An object which produces an iterator via `iter(obj)`
2. _iterator_
    * Calling `next(i)` will produce next object in the series (until you reach the end of the sequence and a `StopIteration` exception is raised)
    * Calling `iter` on an iterator just returns the object itself

The _range_ object is a kind of iterable

In [16]:
x = range(8, -9, -2) # create range object that iterates through 8, 6, 4, 2, 0, −2, −4, −6, −8
y=iter(x)
print('next(y)={}'.format(next(y)))
z = iter(y)
print('next(z)={}'.format(next(z))) #  notice how iter copies iterator completely (including index)

next(y)=8
next(z)=6


range also implements `__len__` and `__getitem__` (so can iterate through it implicitly)

In [17]:
x=range(10)
print(x[2])
print(len(x))

2
10


`Generators` are functions which produce iterators

In [18]:
def factors(n): # generator that computes factors
    k=1
    while k*k < n: # while k < sqrt(n)
        if n % k == 0:
            yield k
            yield n // k
        k += 1
    if k*k == n: # special case if n is perfect square
        yield k

In [19]:
x=factors(21)

iterx = iter(x)

for _ in range(4):
    print(next(iterx))

1
21
3
7


Different kinds of iterators depending on data structure of iterable (list, dict, set, ...)

In [20]:
x=iter([1,2,3])
y=iter({1,2,3})
z=iter({'1':2, '2':1})
print('type of x is {}'.format(type(x)))
print('type of y is {}'.format(type(y)))
print('type of z is {}'.format(type(z)))

type of x is <class 'list_iterator'>
type of y is <class 'set_iterator'>
type of z is <class 'dict_keyiterator'>


Notice that the iterator references the underlying iterable. So changes to the iterable will also change the values returned by the iterator

In [21]:
test_list = [1,2,3]
test_iterator = iter(test_list)

test_list[2] = 4                                                # changing element AFTER creating iterator from it!
print('test list values: {}'.format(test_list))

print('values returned by iterator on test list: [', end='')
for i in test_iterator:
    print('{}'.format(i), end=' ')                              # notice that test_iterator has the same change
print(']')

test list values: [1, 2, 4]
values returned by iterator on test list: [1 2 4 ]


Same with sets!

In [22]:
test_set = {1,2,3}
test_iterator = iter(test_set)

test_set.remove(2)
test_set.add(4)

print('test set values: {}'.format(test_set))

print('values returned by iterator on test set: {', end='')
for i in test_iterator:
    print('{}'.format(i), end=' ')
print('}')

test set values: {1, 3, 4}
values returned by iterator on test set: {1 3 4 }


Notice here that mutating the entry of the list doesn't change anything, this is because int is not _mutable_

In [23]:
test_list = [3,2,1]
test_iterator = iter(test_list)

print('values returned by iterator on test list: [', end='')
for item in test_iterator:
    if item == 1:
        item += 1                              # wont change outside scope
        assert(id(item)==id(test_list[1]))     # these point to the same obect!
    print('{}'.format(item), end=' ') 
print(']')

print('test list values: {}'.format(test_list))

values returned by iterator on test list: [3 2 2 ]
test list values: [3, 2, 1]


Lists are mutatable though, so the underlying iterable will have its values changed!

In [24]:
test_list = [[3,2,1],['hi','hello']]                          # list of lists (crucially, the inner lists are mutable!)
test_iterator = iter(test_list)

print('values returned by iterator on test list: [', end='')
for item in test_iterator:
    if item == ['hi','hello']:
        item[0] = 'changed'                                    # mutating list
    print('{}'.format(item), end=' ') 
print(']')

print('test list values: {}'.format(test_list))

values returned by iterator on test list: [[3, 2, 1] ['changed', 'hello'] ]
test list values: [[3, 2, 1], ['changed', 'hello']]


## Scopes and Namespaces

namespace of function has priority over namespace at top level

In [25]:
def set_x(x):
    print('inner namespace x={}'.format(vars()['x'])) # 1

x=0
set_x(1)
print('outer namespace x={}'.format(vars()['x'])) # 0


inner namespace x=1
outer namespace x=0


Passing by reference vs passing by value vs what Python does.
(this nifty example illustrates the difference (taken from [here](https://robertheaton.com/2014/02/09/pythons-pass-by-object-reference-as-explained-by-philip-k-dick/)))

In Python functions: "Object references are passed by value"

What does this mean? In the example below, the reference to 

In [26]:
def reassign(_list_ref):
    # in pass-by-reference, _list would change here since _list and _list_ref point to the same object
    # no effect if pass-by-value, since _list_ref and _list point to different objects
    _list_ref = [0, 1]

def append(_list_ref):
    # in pass-by-reference, _list would change for same reason as above
    # no effect if pass-by-value, for same reason as above
    _list_ref.append(1)

_list = [0]
reassign(_list)
print('_list is: {}'.format(_list))

append(_list)
print('_list is: {}'.format(_list))

# pass-by-reference would give [0,1,1]
# pass-by-value would give [0]

_list is: [0]
_list is: [0, 1]


## Loading modules

We can import a python script as a module or execute it using the interpreter.
We can hide certain code depending on how we run it.

`module.py` contains the following code:
```
print('loaded as module or script')

if __name__ == '__main__':
    print('loaded as script') # this only runs if its a script
```

In [27]:
import module

loaded as module or script


In [28]:
exec(open('module.py').read())

loaded as module or script
loaded as script
